In [ ]:
import numpy as np
import polars as pl

import lightgbm as lgb
import dehb as DEHB
import ConfigSpace as CS

import time
import math
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# define fidelity range
min_budget, max_budget = 50, 5000

# define config space


def create_search_space(seed=0):
    """Parameter space to be optimized --- contains the hyperparameters"""
    cs = CS.ConfigurationSpace(seed=seed)

    cs.add_hyperparameters([
        # CS.UniformFloatHyperparameter('learning_rate', lower=1e-4, upper=1.0, default_value=1e-2, log=True),
        # CS.UniformFloatHyperparameter('lambda_l1', lower=1e-4, upper=10.0, default_value=1e-4, log=True),
        # CS.UniformFloatHyperparameter('lambda_l2', lower=1e-4, upper=10.0, default_value=1e-4, log=True),
        # CS.UniformFloatHyperparameter('feature_fraction', lower=0.6, upper=1.0, default_value=1.0, log=False),
        # CS.UniformFloatHyperparameter('bagging_fraction', lower=0.6, upper=1.0, default_value=1.0, log=False),
        # CS.UniformIntegerHyperparameter('num_leaves', lower=16, upper=256, default_value=32, log=False),
        CS.UniformFloatHyperparameter('cat_l2', lower=5.0, upper=15.0, default_value=10.0, log=False),
        CS.UniformFloatHyperparameter('cat_smooth', lower=5.0, upper=15.0, default_value=10.0, log=False)
    ])

    return cs


cs = create_search_space(seed=0)
print(cs)

dimensions = len(cs.get_hyperparameters())
print("Dimensionality of search space: {}".format(dimensions))

Configuration space object:
  Hyperparameters:
    bagging_fraction, Type: UniformFloat, Range: [0.6, 1.0], Default: 1.0
    cat_l2, Type: UniformFloat, Range: [5.0, 15.0], Default: 10.0
    cat_smooth, Type: UniformFloat, Range: [5.0, 15.0], Default: 10.0
    feature_fraction, Type: UniformFloat, Range: [0.6, 1.0], Default: 1.0
    num_leaves, Type: UniformInteger, Range: [16, 256], Default: 32

Dimensionality of search space: 5


In [ ]:
# define eval - earlys - valid data sets
evalearlys_data_x, valid_data_x, evalearlys_data_y, valid_data_y = train_test_split(
    train_data_x,
    train_data_y,
    train_size=0.9,
    random_state=0
)

eval_data_x, earlys_data_x, eval_data_y, earlys_data_y = train_test_split(
    evalearlys_data_x,
    evalearlys_data_y,
    train_size=0.9,
    random_state=0
)

lgb_data_eval = lgb.Dataset(data=eval_data_x, label=eval_data_y)
lgb_data_earlys = lgb.Dataset(
    data=earlys_data_x, label=earlys_data_y, reference=lgb_data_eval)
lgb_data_valid = lgb.Dataset(
    data=valid_data_x, label=valid_data_y, reference=lgb_data_eval)
lgb_data_test = lgb.Dataset(
    data=test_data_x, label=test_data_y, reference=lgb_data_eval)

In [ ]:
# function interface with DEHB
def target_function(config, budget, **kwargs):
    # extracting support information
    max_budget = kwargs["max_budget"]

    if budget is None:
        budget = max_budget

    params = {
        'objective': 'regression',
        'metric': 'rmse',
        'boosting': 'gbdt',
        'verbosity': -1,
        'seed': 0
    }

    params.update(config.get_dictionary())

    start = time.time()

    gbm = lgb.train(
        params=params,
        num_boost_round=int(budget),
        train_set=lgb_data_eval,
        valid_sets=[lgb_data_earlys],
        callbacks=[lgb.early_stopping(stopping_rounds=10, verbose=False)],
        keep_training_booster=True
    )

    cost = time.time() - start

    valid_data_pred = gbm.predict(valid_data_x, num_iteration=gbm.best_iteration)
    valid_rmse = mean_squared_error(
        valid_data_y, valid_data_pred, squared=False)

    test_data_pred = gbm.predict(test_data_x, num_iteration=gbm.best_iteration)
    test_rmse = mean_squared_error(test_data_y, test_data_pred, squared=False)

    best_iteration = gbm.best_iteration
    
    result = {
        "fitness": valid_rmse,  
        "cost": cost,
        "info": {
            "test_score": test_rmse,
            "budget": budget, 
            "best_iteration": best_iteration
        }
    }

    return result

In [ ]:
# define tuning algorithm (hyper-hyperparams are default)
dehb = DEHB(
    f=target_function,
    cs=cs,
    dimensions=dimensions,
    min_budget=min_budget,
    max_budget=max_budget,
    eta=3, 
    strategy='rand1_bin',
    mutation_factor=0.5,
    crossover_prob=0.5,
    n_workers=4,
    output_path="./temp"
)

In [ ]:
# run tuning algorithm
dehb.reset()

trajectory, runtime, history = dehb.run(
    total_cost=None, 
    fevals=100,
    verbose=False,
    max_budget=dehb.max_budget, 
)

2023-04-23 07:35:22.872 | INFO     | dehb.optimizers.dehb:reset:107 - 

RESET at 04/23/23 07:35:22 W. Europe Summer Time




2023-04-23 07:35:23,105 - distributed.nanny - WARNING - Restarting worker
2023-04-23 07:35:23,124 - distributed.nanny - WARNING - Restarting worker
2023-04-23 07:35:23,131 - distributed.nanny - WARNING - Restarting worker
2023-04-23 07:35:23,138 - distributed.nanny - WARNING - Restarting worker


In [ ]:
print(len(trajectory), len(runtime), len(history), end="\n\n")

# last recorded function evaluation
last_eval = history[-1]
config, score, cost, budget, _info = last_eval

print("Last evaluated configuration, ")
print(dehb.vector_to_configspace(config), end="")
print("got a score of {}, was evaluated at a budget of {:.2f} and "
      "took {:.3f} seconds to run.".format(score, budget, cost))
print("The additional info attached: {}".format(_info), end="\n\n")


print("Best evaluated configuration, ")
print(dehb.vector_to_configspace(dehb.inc_config), end="")
print("got a score of {}.".format(dehb.inc_score))
print("The additional info attached: {}".format(dehb.inc_info), end="\n\n")

212 212 212

Last evaluated configuration, 
Configuration(values={
  'bagging_fraction': 0.6340546324562695,
  'cat_l2': 9.618534421619371,
  'cat_smooth': 6.806606215406344,
  'feature_fraction': 0.63399434125221,
  'lambda_l1': 8.275173171359791,
  'lambda_l2': 0.0012553755831805511,
  'learning_rate': 0.0001792569563679823,
  'num_leaves': 107,
})
got a score of 10.33612642703613, was evaluated at a budget of 10000.00 and took 481.239 seconds to run.
The additional info attached: {'test_score': 10.123112749160718, 'budget': 10000.0, 'best_iteration': 10000}

Best evaluated configuration, 
Configuration(values={
  'bagging_fraction': 0.8088284912846216,
  'cat_l2': 8.034264947167088,
  'cat_smooth': 9.075255369543438,
  'feature_fraction': 0.522285555493832,
  'lambda_l1': 0.034076843234482956,
  'lambda_l2': 0.0077864425839706776,
  'learning_rate': 0.018365651651009347,
  'num_leaves': 64,
})
got a score of 9.625861223941522.
The additional info attached: {'test_score': 9.568879269